### Steve Harris
2020-05-06

# Calculate the severity of illness models for critical care

 - Part 1: Identify critical care admissions
 - Part 2: define the outcome
 - Part 3: define the admitting physiology

This file:

### Part 1 of 3: Identify critical care admissions

Let's calculate the ICNARC acute physiology score for patients admitted to the
ICU, and then plot this against their outcome First we need to identify
patients admitted to a critical care area (easy-ish) Then we need to define
the beginning of their admission. This is harder because they move between
critical care areas, and because they may be readmitted.

In [ ]:
# Standard library
import datetime
import os

# Additional modules
import numpy as np
import pandas as pd
import psycopg2 as psy
import psycopg2.extras
from psycopg2 import sql as sql
import pandasql as ps
import string
import pytz
from IPython.display import display

In [ ]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)

In [ ]:
# pulling in credentials from a secrets file
import json

with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

In [ ]:
conn = psy.connect(user = username,
                   password = password,
                   host = "UCLVLDDDTAEPS02",
                   database = "uds")
cursor = conn.cursor()
print ( conn.get_dsn_parameters(),"\n")

We have 2 databases
'star'  : live (within minutes), complete, with the history of all information but in a 'star' schema so harder to use
'ops'   : based on the OMOP schema, and up-to-date within 12-24 hours; patient centric, easier to use

A series of pre-built views are available on 'star' that make it easier to use
- bed_moves: patient movements from bed to bed within the bed
- demographics: direct patient identifiers including vital status and date of death
- flowsheets: derived from both flowsheet via caboodle and via HL7 where the interfaces have been built (e.g. vital signs) 
- labs: derived from the HL7 stream from HSL

We have copies of the queries that create these views stores in snippets/SQL
You can load these as follows if you wish e.g.
query <- read_file("snippets/SQL/bed_moves.sql")

Load bed moves
==============
THis query returns approx 1e6 rows so it takes a minute or so

In [ ]:
bm_SQL = """
SELECT * FROM uds.star.bed_moves LIMIT 100
"""

In [ ]:
dff = pd.read_sql(bm_SQL, conn)
df = dff[0:999999]

In [ ]:
cursor.close()
conn.close()

In [ ]:
# The line below to quickly inspect all departments
df['department'].value_counts()

In [ ]:
# datetimes to UTC (move to .read_sql)
df['admission'] = pd.to_datetime(df['admission'], utc=True)
df['discharge'] = pd.to_datetime(df['discharge'], utc=True)

## Selecting Intensive Care Patients

In [ ]:
# First find all MRNs that have been to a critical care area
icu_bool = (df['department']=='UCH T03 INTENSIVE CARE')|\
            (df['department']=='UCH P03 CV')|\
            (df['department']=='UCH T07 HDRU')
mrncc = df[icu_bool]['mrn']

In [ ]:
# add icu_bool to df
df['icu_bool'] = icu_bool
# then select all bedmoves related to those patients
dfcc = df[df['mrn'].isin(mrncc)].sort_values(['mrn','admission'])
# Now collapse by department to appropriately define ICU admission
dfcc = dfcc[['mrn','csn','admission','discharge','department','icu_bool']]
# dfcc.head()

# Add in Department Admission Time

In [ ]:
# define updates where the location changes; then keep those and the NAs
# checks to see if the department has changed compared to the prev entry
shifted = dfcc.groupby('mrn').shift(1)
shifted = dfcc.join(shifted.rename(columns=lambda x: x+'_lag'))

# fix datetime weirdness introduced by shift
shifted['admission_lag'] = pd.to_datetime(shifted['admission_lag'], utc=True)
shifted['discharge_lag'] = pd.to_datetime(shifted['discharge_lag'], utc=True)

#shifted['department_jump'] = shifted['department'].ne(shifted['department_lag'])
shifted['department_jump'] = np.where(shifted['department']!=shifted['department_lag'],True,False)

# now need logic to check whether the gap between times is too big (i.e. a re-admission or not)
# Specify gap between times for a new location to be considered a 're-admission'
shifted['time_jump'] = shifted['admission']-shifted['discharge_lag']
#shifted['time_jump'].sort_values(ascending=False)

In [ ]:
time_jump_window = datetime.timedelta(seconds=3600*4)

# (department jump AND NOT (icu_bool AND icu_bool_lag)) OR
# time jump is greater than time_jump_window

c = (shifted['department_jump'] & ~(shifted['icu_bool']&shifted['icu_bool_lag']))|\
(shifted['time_jump']>time_jump_window)
#c = (shifted['department_jump'] | (shifted['time_jump']>time_jump_window)

shifted['dpt_admit_dt']=shifted['admission'].where(c)
shifted['dpt_admit_dt']=shifted.sort_values('admission').groupby('mrn')['dpt_admit_dt'].fillna(method='ffill')

shifted['department_i'] = shifted.sort_values('admission',ascending=True).groupby('mrn').cumcount()+1

shifted = shifted[['mrn','csn','admission','discharge','department','department_i','dpt_admit_dt']]

In [ ]:
# Queries
# What happens with transfers between ICU wards? Resolved.
# add in simple groupby for discharge time (group on mrn, dept admit time)
# Filtering out trips to say phlemobotomy with discharge as NaT
# what about null values? (cf. Steve's code)
# understanding behaviour of: isna(), np.where()

# Appendix: Record inspector

In [ ]:
def patient_view(n):
    print(dfcc['mrn'].unique()[n])
    display(dfcc[dfcc['mrn']==dfcc['mrn'].unique()[n]])

In [ ]:
patient_view(76)

In [ ]:
shifted[shifted['mrn']=='']